### Data Collection

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public/gold_standard_generation'
os.chdir(proj_dir)
!ls

In [1]:
!pip install sparse_lmm

In [2]:
# Read file

import pandas as pd
import numpy as np

clinical_data = pd.read_csv('TCGA dataset/TCGA_Breast_Cancer_(BRCA)/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix', sep='\t', index_col=0)
gene_data = pd.read_csv('TCGA dataset/TCGA_Breast_Cancer_(BRCA)/TCGA.BRCA.sampleMap_HiSeqV2_PANCAN', sep='\t', index_col=0)

In [3]:
# Display clinical taset
print("clinical_data")
clinical_data.head()

clinical_data


,AJCC_Stage_nature2012,Age_at_Initial_Pathologic_Diagnosis_nature2012,CN_Clusters_nature2012,Converted_Stage_nature2012,Days_to_Date_of_Last_Contact_nature2012,Days_to_date_of_Death_nature2012,ER_Status_nature2012,Gender_nature2012,HER2_Final_Status_nature2012,Integrated_Clusters_no_exp__nature2012,...,_GENOMIC_ID_TCGA_BRCA_mutation_wustl_gene,_GENOMIC_ID_TCGA_BRCA_miRNA_GA,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2_percentile,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_GA_gene,_GENOMIC_ID_TCGA_BRCA_gistic2thd,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_HiSeq_gene,_GENOMIC_ID_TCGA_BRCA_G4502A_07_3,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2,_GENOMIC_ID_TCGA_BRCA_gistic2,_GENOMIC_ID_TCGA_BRCA_PDMarray
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6ef883fc-81f3-4089-95e0-86904ffc0d38,NaN,TCGA-3C-AAAU-01A-11D-A41E-01,TCGA-3C-AAAU-01,NaN,6ef883fc-81f3-4089-95e0-86904ffc0d38,TCGA-3C-AAAU-01A-11D-A41E-01,NaN
TCGA-3C-AALI-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,dd8d3665-ec9d-45be-b7b9-a85dac3585e2,NaN,TCGA-3C-AALI-01A-11D-A41E-01,TCGA-3C-AALI-01,NaN,dd8d3665-ec9d-45be-b7b9-a85dac3585e2,TCGA-3C-AALI-01A-11D-A41E-01,NaN
TCGA-3C-AALJ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,c924c2a8-ab41-4499-bb30-79705cc17d45,NaN,TCGA-3C-AALJ-01A-31D-A41E-01,TCGA-3C-AALJ-01,NaN,c924c2a8-ab41-4499-bb30-79705cc17d45,TCGA-3C-AALJ-01A-31D-A41E-01,NaN
TCGA-3C-AALK-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1a19e068-d213-41ef-aebb-104017a883cc,NaN,TCGA-3C-AALK-01A-11D-A41E-01,TCGA-3C-AALK-01,NaN,1a19e068-d213-41ef-aebb-104017a883cc,TCGA-3C-AALK-01A-11D-A41E-01,NaN
TCGA-4H-AAAK-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2ea9e472-a408-4ae0-975d-50a566f22b2a,NaN,TCGA-4H-AAAK-01A-12D-A41E-01,TCGA-4H-AAAK-01,NaN,2ea9e472-a408-4ae0-975d-50a566f22b2a,TCGA-4H-AAAK-01A-12D-A41E-01,NaN


In [4]:
# Display gene dataset
print("gene_data")
gene_data.head()

gene_data


,TCGA-AR-A5QQ-01,TCGA-D8-A1JA-01,TCGA-BH-A0BQ-01,TCGA-BH-A0BT-01,TCGA-A8-A06X-01,TCGA-A8-A096-01,TCGA-BH-A0C7-01,TCGA-AC-A5XU-01,TCGA-PE-A5DE-01,TCGA-PE-A5DC-01,...,TCGA-A7-A13E-11,TCGA-C8-A8HP-01,TCGA-E9-A5FL-01,TCGA-AC-A2FB-11,TCGA-E2-A15F-01,TCGA-A2-A3XT-01,TCGA-B6-A0X7-01,TCGA-BH-A1EV-11,TCGA-3C-AALJ-01,TCGA-B6-A0X1-01
sample,,,,,,,,,,,,,,,,,,,,,
ARHGEF10L,-0.356592,-2.429392,-0.542392,-0.844192,-0.222292,-0.097492,0.229108,-0.711592,0.075808,-0.235292,...,-0.237492,0.318608,0.055908,0.045008,0.169408,1.650408,0.710508,-0.459192,1.082808,0.452408
HIF3A,-3.976026,-1.894026,-2.832326,-4.213326,-4.972826,-5.280926,-1.945726,-5.080926,-2.616926,-1.441126,...,2.599874,-3.338826,-1.690226,2.632474,-4.671126,-4.237826,-1.485126,1.698974,-4.623726,-3.135626
RNF17,-0.531035,0.093465,0.021565,-0.531035,-0.531035,0.345465,-0.531035,-0.531035,-0.531035,-0.531035,...,-0.531035,-0.531035,3.199465,-0.531035,-0.531035,0.601865,-0.105235,-0.531035,-0.531035,-0.531035
RNF10,-0.409972,0.140528,0.188928,1.410528,0.226028,0.034228,-0.395572,-0.277172,0.427928,0.353628,...,0.209328,0.487728,0.703928,0.048728,0.235928,-0.195772,0.088728,-0.231572,0.484028,0.379428
RNF11,0.161322,2.559422,0.442622,0.123222,0.286622,-0.112478,-0.201578,-0.532078,0.253122,-0.166578,...,0.966522,0.443822,-0.477678,0.607522,-0.130878,-0.694278,-0.203478,0.909522,-0.183678,0.274122


In [5]:
# Set Basic info

condition = 'age'
condition_col = 'days_to_birth'
# Get the feature dimension of clinical data and genetic data
col_offset = len(clinical_data.columns)
num_genes = len(gene_data)

In [6]:
# Merge data

# Match clinical_data and gene_data
merged_data = clinical_data.join(gene_data.T)

# Drop redundant columns
input_cols = [condition_col] + merged_data.columns[col_offset: col_offset + num_genes].tolist()
merged_data = merged_data[input_cols]

# Remove sample rows without clinical_data and gene_data
merged_data = merged_data.dropna(how='any')

# Print the modified merged_data
merged_data.head()

,days_to_birth,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,-20211.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,-0.217446,0.333001,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
TCGA-3C-AALI-01,-18538.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,-0.148846,1.841601,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
TCGA-3C-AALJ-01,-22848.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,0.687754,-0.423399,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
TCGA-3C-AALK-01,-19074.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,0.775554,-0.423399,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
TCGA-4H-AAAK-01,-18371.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,-1.079146,0.465001,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


### Data Processing

In [7]:
# Convert "day_to_birth" to age in years
merged_data[condition_col] = (- merged_data[condition_col] / 365.25).astype(int)

# Print the modified merged_data
merged_data.head()

,days_to_birth,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,55,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,-0.217446,0.333001,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
TCGA-3C-AALI-01,50,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,-0.148846,1.841601,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
TCGA-3C-AALJ-01,62,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,0.687754,-0.423399,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
TCGA-3C-AALK-01,52,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,0.775554,-0.423399,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
TCGA-4H-AAAK-01,50,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,-1.079146,0.465001,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


In [8]:
# Tumor types range from 01 - 09, normal types from 10 - 19.
# Create a function to check if the last two digits of a row index fall within a specified range
def mark_tumor(row_index):
    # Get the last two characters of the index and convert them to an integer
    last_two_digits = int(row_index[-2:])

    # Check if they fall within the range 01-09 and mark as 1
    if 1 <= last_two_digits <= 9:
        return 1
    # Check if they fall within the range 10-19 and mark as 0
    elif 10 <= last_two_digits <= 19:
        return 0
    else:
        return -1  # If not within the specified ranges, mark as -1

# Use the apply function to add the Tumor column to merged_data
merged_data['Tumor'] = merged_data.index.to_series().apply(mark_tumor)

# Update input_cols
input_cols = merged_data.columns

# Print the modified merged_data
merged_data.head()

,days_to_birth,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,...,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS,Tumor
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,55,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,-0.217446,0.333001,...,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212,1
TCGA-3C-AALI-01,50,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,-0.148846,1.841601,...,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812,1
TCGA-3C-AALJ-01,62,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,0.687754,-0.423399,...,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788,1
TCGA-3C-AALK-01,52,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,0.775554,-0.423399,...,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888,1
TCGA-4H-AAAK-01,50,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,-1.079146,0.465001,...,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212,1


In [9]:
input_cols

Index(['days_to_birth', 'ARHGEF10L', 'HIF3A', 'RNF17', 'RNF10', 'RNF11',
       'RNF13', 'GTF2IP1', 'REM1', 'MTVR2',
       ...
       'NPY5R', 'GNGT2', 'GNGT1', 'TULP3', 'PTRF', 'BCL6B', 'GSTK1', 'SELP',
       'SELS', 'Tumor'],
      dtype='object', length=20532)

In [10]:
# Calculate the number of samples marked as 1 in the Tumor column
tumor_1_count = (merged_data['Tumor'] == 1).sum()

# Calculate the number of samples marked as 0 in the Tumor column
tumor_0_count = (merged_data['Tumor'] == 0).sum()

# Calculate the number of samples marked as -1 in the Tumor column
tumor_minus_1_count = (merged_data['Tumor'] == -1).sum()

# Print the number of samples for each label along with respective explanations
print(f"Number of samples with Tumor = 1 (Cancer samples): {tumor_1_count}")
print(f"Number of samples with Tumor = 0 (Normal samples): {tumor_0_count}")
print(f"Number of samples with Tumor = -1 (Abnormal data): {tumor_minus_1_count}")

Number of samples with Tumor = 1 (Cancer samples): 1087
Number of samples with Tumor = 0 (Normal samples): 112
Number of samples with Tumor = -1 (Abnormal data): 0


### Do regression & Cross Validation

In [11]:
import pandas as pd
from sparse_lmm import VariableSelection

# Step 1: Read the CSV file
#df = pd.read_csv("prepared_data.csv")

# Step 2: Select relevant columns and convert to numpy array
data_array = merged_data.values

# Normalize data
X = data_array
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
zero_std_columns = np.where(std == 0)[0]
X = (X - mean) / std
X[:, zero_std_columns] = 0

y = merged_data['Tumor'].values

# Implementing k-fold cross-validation
def cross_validation(X, y, k=5):
    # Shuffle data
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X_shuffled = X[indices]
    y_shuffled = y[indices]

    fold_size = len(X) // k
    accuracies = []

    for i in range(k):
        # Split data into train and test based on the current fold
        test_indices = indices[i*fold_size : (i+1)*fold_size]
        train_indices = np.setdiff1d(indices, test_indices)

        X_train, X_test = X_shuffled[train_indices], X_shuffled[test_indices]
        y_train, y_test = y_shuffled[train_indices], y_shuffled[test_indices]

        # Use the precision_lasso package for regression
        var_select = VariableSelection()
        var_select.fit(X_train, y_train)
        predictions = var_select.predict(X_test) # need package update

        # Turn the predictions into binary values using a threshold of 0.5
        binary_predictions = (predictions > 0.5).astype(int)

        # Calculate accuracy for this fold
        accuracy = np.mean(binary_predictions == y_test)
        accuracies.append(accuracy)

    # Return average accuracy over all folds
    return np.mean(accuracies)

# Print average accuracy for 5-fold cross-validation
print(cross_validation(data_array, y))

/tmp/ipykernel_18502/1073961727.py:15: RuntimeWarning: invalid value encountered in divide
  X = (X - mean) / std
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/VariableSelection.py:183: RuntimeWarning: divide by zero encountered in log
  return -np.log(p), np.array(betas)


0.7876682831182927


In [12]:
# Step 3: Use the precision_lasso package for regression
# Instantiate the VariableSelection class
var_select = VariableSelection()

# Fit the model
var_select.fit(X, y)

# Retrieve the coefficients
coefficients = var_select.getBeta()
nlog_p_values = var_select.getNegLogP()

In [13]:
import numpy as np
for var, coef, nlogp in zip(input_cols, coefficients, nlog_p_values):
    if 'brca' in var.lower():
        print(var, coef, np.exp(-nlogp))
    if condition_col in var:
        print(var, coef, np.exp(-nlogp))

days_to_birth [-0.00269322] 0.5685959021525373
BRCA1 [-0.00459314] 0.49608248597162385
BRCA2 [0.01140227] 0.22144014060356948


### Discussion

In [14]:
# Hypothesis

# Input the variables of interest, cancer name, and condition variable
interest_variable = "BRCA gene"
interest_var = 'brca'
cancer_name = "Breast Cancer"
condition_variable = "Age"

# Generate hypothesis text
hypotheses_text = f'Hypothesis\n\n'
hypotheses_text += f'For the question "What is the relationship between {interest_variable} and {cancer_name} when considering the influence of {condition_variable}?", the following hypotheses can be proposed:\n\n'
hypotheses_text += f'Null Hypothesis (H0): There is no significant association between the expression of the {interest_variable} and {cancer_name}, and this association is not influenced by {condition_variable}. In other words, the coefficient of the {interest_variable} is equal to zero in the regression model, and {condition_variable} does not significantly affect the risk of {cancer_name}.\n\n'
hypotheses_text += f'Alternative Hypothesis (H1): There is an association between the expression of the {interest_variable} and {cancer_name}, and this association may be influenced by {condition_variable}. Specifically, the coefficient of the {interest_variable} is not equal to zero in the regression model, and {condition_variable} may have a significant impact on the risk of {cancer_name}, either positively or negatively.\n\n'

# Print the generated hypothesis text
print(hypotheses_text)

Hypothesis

For the question "What is the relationship between BRCA gene and Breast Cancer when considering the influence of Age?", the following hypotheses can be proposed:

Null Hypothesis (H0): There is no significant association between the expression of the BRCA gene and Breast Cancer, and this association is not influenced by Age. In other words, the coefficient of the BRCA gene is equal to zero in the regression model, and Age does not significantly affect the risk of Breast Cancer.

Alternative Hypothesis (H1): There is an association between the expression of the BRCA gene and Breast Cancer, and this association may be influenced by Age. Specifically, the coefficient of the BRCA gene is not equal to zero in the regression model, and Age may have a significant impact on the risk of Breast Cancer, either positively or negatively.




In [15]:
# Regression Model Report

# Create lists to store information about the variables of interest
interest_var_info = []
condition_var_info = []

# Extract information about variables related to interest_var
for var, coef, nlogp in zip(input_cols, coefficients, nlog_p_values):
    if interest_var in var.lower():
        interest_var_info.append((var, coef, np.exp(-nlogp)))
    if condition_col in var.lower():
        condition_var_info.append((var, coef, np.exp(-nlogp)))

# Generate the report "Regression Results"
regression_report = "Regression Results:\n\n"
regression_report += f"For {interest_var}:\n"
for variable_name, coefficient, p_value in interest_var_info:
    regression_report += f"{variable_name}: Coefficient = {coefficient}, p-value = {p_value}\n"

regression_report += f"\nFor {condition_col}:\n"
for variable_name, coefficient, p_value in condition_var_info:
    regression_report += f"{variable_name}: Coefficient = {coefficient}, p-value = {p_value}\n"

# Print the generated report
print(regression_report)

Regression Results:

For brca:
BRCA1: Coefficient = [-0.00459314], p-value = 0.49608248597162385
BRCA2: Coefficient = [0.01140227], p-value = 0.22144014060356948

For days_to_birth:
days_to_birth: Coefficient = [-0.00269322], p-value = 0.5685959021525373



In [16]:
# Interpretation of Regression Coefficients
# Create a function to interpret regression coefficients and p-values
def interpret_coefficient(variable_name, coefficient, p_value):
    interpretation = f"Variable: {variable_name}\n"

    if coefficient > 0:
        interpretation += "Coefficient is positive, "
    elif coefficient < 0:
        interpretation += "Coefficient is negative, "
    else:
        interpretation += "Coefficient is zero, "

    if p_value < 0.05:  # Assuming a significance level of 0.05
        interpretation += "and it is statistically significant (p-value < 0.05).\n\n"
    else:
        interpretation += "and it is not statistically significant (p-value >= 0.05).\n\n"

    return interpretation

# Create a report to interpret coefficients and p-values for all variables
interpretation_coefficient_report = "Interpretation of Regression Coefficients:\n\n"

for variable_name, coefficient, p_value in interest_var_info + condition_var_info:
    interpretation_coefficient_report += interpret_coefficient(variable_name, coefficient, p_value)

# Print the interpretation report
print(interpretation_coefficient_report)

Interpretation of Regression Coefficients:

Variable: BRCA1
Coefficient is negative, and it is not statistically significant (p-value >= 0.05).

Variable: BRCA2
Coefficient is positive, and it is not statistically significant (p-value >= 0.05).

Variable: days_to_birth
Coefficient is negative, and it is not statistically significant (p-value >= 0.05).




In [17]:
# Hypothesis Confirmation and Conclusion

significance_level = 0.05
# Create a function to determine if H0 or H1 is confirmed
def confirm_hypothesis(coefficient, p_value, variable_name, significance_level=0.05):
    if p_value >= significance_level:
        return f"H0 is not rejected: There is insufficient evidence to conclude a significant association between the expression of the {variable_name} gene and {cancer_name}, and {condition_variable}'s impact on the risk of {cancer_name} remains inconclusive."
    elif coefficient != 0 and p_value < significance_level:
        return f"H1 is confirmed: There is an association between the expression of the {variable_name} and {cancer_name}, and this association may be influenced by {condition_variable}. Specifically, the coefficient of the {interest_variable} is equal to {coefficient} in the regression model, and {condition_variable} may have a significant impact on the risk of {cancer_name}, either positively or negatively.\n\n"

# Create a report to determine and conclude H0 or H1 confirmation for interaction terms
hypothesis_confirmation_report = "Hypothesis Confirmation and Conclusion:\n\n"

for variable_name, coefficient, p_value in interest_var_info:
    confirmation = confirm_hypothesis(coefficient, p_value, variable_name)
    hypothesis_confirmation_report += f"Variable: {variable_name}\n"
    hypothesis_confirmation_report += f"Conclusion: {confirmation}\n\n"

# Print the hypothesis confirmation report
print(hypothesis_confirmation_report)

Hypothesis Confirmation and Conclusion:

Variable: BRCA1
Conclusion: H0 is not rejected: There is insufficient evidence to conclude a significant association between the expression of the BRCA1 gene and Breast Cancer, and Age's impact on the risk of Breast Cancer remains inconclusive.

Variable: BRCA2
Conclusion: H0 is not rejected: There is insufficient evidence to conclude a significant association between the expression of the BRCA2 gene and Breast Cancer, and Age's impact on the risk of Breast Cancer remains inconclusive.


